In [ ]:
import pandas as pd
from google.colab import files

class NodoTrie():
    """Clase que representa un nodo en el trie."""

    def __init__(self):
        """Inicializa un nodo trie con un diccionario de hijos y una letra."""
        self.hijos = {}
        self.letra = None

class Trie():
    """Clase que representa el trie y contiene métodos para construirlo, codificar y decodificar."""

    def __init__(self, archivo_excel):
        """
        Inicializa el trie y lo construye usando un archivo de Excel.

        Args:
            archivo_excel (str): El nombre del archivo de Excel que contiene las columnas 'Letras' y 'Cadenas'.
        """
        self.raiz = NodoTrie()
        self.construir_trie(archivo_excel)

    def construir_trie(self, archivo_excel):
        """
        Construye el trie leyendo un archivo de Excel y añadiendo las letras y sus cadenas binarias correspondientes.

        Args:
            archivo_excel (str): El nombre del archivo de Excel.

        Raises:
            ValueError: Si el archivo no contiene las columnas 'Letras' y 'Cadenas', o si hay valores nulos o cadenas binarias inválidas.
        """
        df = pd.read_excel(archivo_excel)
        # Verificación de las columnas esperadas
        if 'Letras' not in df.columns or 'Cadenas' not in df.columns:
            raise ValueError("El archivo Excel debe contener las columnas 'Letras' y 'Cadenas'")

        for _, fila in df.iterrows():
            letra = fila['Letras']
            cadena_binaria = str(fila['Cadenas'])

            # Verificación de datos nulos o vacíos
            if pd.isnull(letra) or pd.isnull(cadena_binaria):
                raise ValueError("Las columnas 'Letras' y 'Cadenas' no deben contener valores nulos o vacíos")

            # Verificación de formato de la cadena binaria
            if not all(bit in '01' for bit in cadena_binaria):
                raise ValueError(f"La cadena binaria '{cadena_binaria}' contiene caracteres no válidos")

            # Mensaje de depuración
            print(f"Insertando la letra '{letra}' con la cadena binaria '{cadena_binaria}' en el trie.")
            self.insertar(letra, cadena_binaria)

    def insertar(self, letra, cadena_binaria):
        """
        Inserta una letra en el trie siguiendo su cadena binaria.

        Args:
            letra (str): La letra a insertar.
            cadena_binaria (str): La cadena binaria correspondiente a la letra.
        """
        nodo = self.raiz
        for bit in cadena_binaria:
            if bit not in nodo.hijos:
                nodo.hijos[bit] = NodoTrie()
            nodo = nodo.hijos[bit]
        nodo.letra = letra
        print(f"Letra '{letra}' insertada con la cadena binaria '{cadena_binaria}'.")

    def codificar(self, texto):
        """
        Codifica un texto en un número binario usando el trie.

        Args:
            texto (str): El texto a codificar.

        Returns:
            int: El número binario que representa el texto codificado.

        Raises:
            ValueError: Si algún carácter del texto no se encuentra en el trie.
        """
        resultado = []
        for caracter in texto:
            # Buscar la codificación binaria del carácter actual
            caracter_codificado = self.buscar_codificacion(self.raiz, caracter, "")
            if caracter_codificado is None:
                raise ValueError(f"El carácter '{caracter}' no se encuentra en el trie.")
            resultado.append(caracter_codificado)
        # Unir todas las cadenas binarias y convertirlas a un entero
        return int(''.join(resultado), 2)

    def buscar_codificacion(self, nodo, caracter, camino):
        """
        Busca recursivamente la codificación binaria de un carácter en el trie.

        Args:
            nodo (NodoTrie): El nodo actual del trie.
            caracter (str): El carácter a buscar.
            camino (str): La cadena de bits acumulada hasta el nodo actual.

        Returns:
            str or None: La cadena de bits que representa el carácter o None si no se encuentra.
        """
        if nodo.letra == caracter:
            return camino
        for bit, hijo in nodo.hijos.items():
            # Llamada recursiva para continuar buscando en los nodos hijos
            resultado = self.buscar_codificacion(hijo, caracter, camino + bit)
            if resultado:
                return resultado
        return None

    def decodificar(self, numero_binario):
        """
        Decodifica un número binario en un texto usando el trie.

        Args:
            numero_binario (int): El número binario a decodificar.

        Returns:
            str: El texto decodificado.

        Raises:
            ValueError: Si el número binario no representa una codificación válida.
        """
        cadena_binaria = bin(numero_binario)[2:]  # Convertir a cadena binaria sin el prefijo '0b'
        resultado = []
        nodo = self.raiz
        camino_actual = ""
        for bit in cadena_binaria:
            camino_actual += bit
            nodo = nodo.hijos.get(bit)
            if nodo and nodo.letra:
                # Añadir la letra decodificada al resultado y reiniciar el nodo
                resultado.append(nodo.letra)
                nodo = self.raiz  # Comenzar de nuevo para la siguiente letra
                camino_actual = ""
        if nodo != self.raiz:  # Verificar si hubo una letra incompleta al final
            raise ValueError("El número binario no representa una codificación válida.")
        return ''.join(resultado)

def main():
    """
    Función principal para ejecutar el programa.

    Permite subir un archivo de Excel y elegir entre codificar o decodificar un texto.
    """
    # Subir archivo de Excel
    subido = files.upload()
    archivo_excel = next(iter(subido))

    # Crear una instancia de Trie usando el archivo subido
    trie = Trie(archivo_excel)

    # Pedir al usuario que elija entre codificar o decodificar
    accion = input("¿Desea codificar o decodificar? (c/d): ").strip().lower()
    if accion == 'c':
        texto = input("Introduce el texto a codificar: ").strip()
        try:
            codigo_binario = trie.codificar(texto)
            print(f"Texto original: {texto}")
            print(f"Código binario: {codigo_binario}")
        except ValueError as e:
            print(f"Error: {e}")
    elif accion == 'd':
        codigo_binario = input("Introduce el código binario a decodificar: ").strip()
        try:
            numero_binario = int(codigo_binario, 2)
            texto_decodificado = trie.decodificar(numero_binario)
            print(f"Código binario: {codigo_binario}")
            print(f"Texto decodificado: {texto_decodificado}")
        except ValueError as e:
            print(f"Error: {e}")
    else:
        print("Acción no reconocida. Por favor, elige 'c' para codificar o 'd' para decodificar.")

main()

Saving Code.xlsx to Code (2).xlsx
Insertando la letra ' ' con la cadena binaria '1' en el trie.
Letra ' ' insertada con la cadena binaria '1'.
Insertando la letra ',' con la cadena binaria '10000110' en el trie.
Letra ',' insertada con la cadena binaria '10000110'.
Insertando la letra '.' con la cadena binaria '1010001' en el trie.
Letra '.' insertada con la cadena binaria '1010001'.
Insertando la letra 'a' con la cadena binaria '1001' en el trie.
Letra 'a' insertada con la cadena binaria '1001'.
Insertando la letra 'b' con la cadena binaria '1010' en el trie.
Letra 'b' insertada con la cadena binaria '1010'.
Insertando la letra 'c' con la cadena binaria '1000001' en el trie.
Letra 'c' insertada con la cadena binaria '1000001'.
Insertando la letra 'd' con la cadena binaria '101010' en el trie.
Letra 'd' insertada con la cadena binaria '101010'.
Insertando la letra 'e' con la cadena binaria '11' en el trie.
Letra 'e' insertada con la cadena binaria '11'.
Insertando la letra 'f' con la c